<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Анализ" data-toc-modified-id="Анализ-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Анализ</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Модель-линейной-регрессии" data-toc-modified-id="Модель-линейной-регрессии-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Модель линейной регрессии</a></span></li><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Дерево решений</a></span></li><li><span><a href="#Модель-Lasso" data-toc-modified-id="Модель-Lasso-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Модель Lasso</a></span></li></ul></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Общий вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

#  Прогнозирование заказов такси

Компания «Чётенькое такси» собрала исторические данные о заказах такси в аэропортах. Чтобы привлекать больше водителей в период пиковой нагрузки, нужно спрогнозировать количество заказов такси на следующий час. Постройте модель для такого предсказания.

Значение метрики *RMSE* на тестовой выборке должно быть не больше 48.

Вам нужно:

1. Загрузить данные и выполнить их ресемплирование по одному часу.
2. Проанализировать данные.
3. Обучить разные модели с различными гиперпараметрами. Сделать тестовую выборку размером 10% от исходных данных.
4. Проверить данные на тестовой выборке и сделать выводы.


Данные лежат в файле `taxi.csv`. Количество заказов находится в столбце `num_orders` (от англ. *number of orders*, «число заказов»).

## Подготовка

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install scikit-learn==1.2.2

In [ ]:
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

In [ ]:
data  = pd.read_csv('/datasets/taxi.csv', parse_dates=['datetime'], index_col=['datetime'])

In [ ]:
data = data.sort_index() 

## Анализ

In [ ]:
def inf(data):
    print(f'Размер датасета - {data.shape}')
    print()
    print('Информация о данных:')
    print(data.info())
    print()
    print(f'Количество пропусков в датасете: {data.isna().sum().sum()}')
    print(f'Указание на пропуски в датасете: {data.isna().sum()}')
    print(f'Дубликаты в датасете: {data.duplicated().sum()}')

In [ ]:
inf(data)

In [ ]:
data.index.min(), data.index.max()

In [ ]:
data = data['2018-03-01 00:00:00':'2018-08-31 23:50:00']
data.plot(color = 'lightcoral',figsize=(20,8))
plt.title('График временного ряда')
plt.xlabel('Дата/время')
plt.ylabel('Кол-во заказов')
plt.show()

In [ ]:
data=data.resample('1H').sum()
data.plot(color = 'rebeccapurple',figsize=(20,8))
plt.title('Распределение количество заказов по времени (ресемплирование по одному часу)')
plt.xlabel('Дата/время')
plt.ylabel('Кол-во заказов')
plt.show()

In [ ]:
data = data.sort_index()
data=data.resample('1H').sum()

**Построим графики временного ряда. Разложим данные на три составляющие: тренд, сезонность и остаток.**

In [ ]:
decomposed = seasonal_decompose(data)

In [ ]:
decomposed.trend.plot(figsize=(20,8))
plt.title('Тренд')
plt.xlabel('Дата/время')
plt.ylabel('Кол-во заказов')
plt.show()

In [ ]:
decomposed.seasonal.plot(figsize=(20,8))
plt.title('Сезонность')
plt.xlabel('Дата/время')
plt.ylabel('Кол-во заказов')
plt.show()

In [ ]:
decomposed.resid.plot(figsize=(20,8))
plt.title('Остаток')
plt.xlabel('Дата/время')
plt.ylabel('Кол-во заказов')
plt.show()

**Рост количества заказов виден летом, что можно объяснить потоком туристов или, наоборот, тех, кто улетает или возвращается с отдыха.**

**Построим графики для последнего и самого потокового месяца временного ряда.**

In [ ]:
som = decomposed.trend['2018-08':'2018-08']
som.plot(color='orange', figsize=(20,8))
plt.title('Тренд')
plt.xlabel('Дата/время')
plt.ylabel('Кол-во заказов')
plt.show()

In [ ]:
nom = decomposed.seasonal['2018-08':'2018-08']
nom.plot(color='magenta', figsize=(20,8))
plt.title('Сезонность')
plt.xlabel('Дата/время')
plt.ylabel('Кол-во заказов')
plt.show()

In [ ]:
tom = decomposed.resid['2018-08':'2018-08']
tom.plot(color='mediumspringgreen', figsize=(20,8))
plt.title('Остаток')
plt.xlabel('Дата/время')
plt.ylabel('Кол-во заказов')
plt.show()

**Построим график временного ряда на неделю начиная с понедельника 13 августа 2018 года**

In [ ]:
week = decomposed.trend['2018-08-13':'2018-08-20']
week.plot(color='palevioletred', figsize=(20,8))
plt.title('Тренд')
plt.xlabel('Дата/время')
plt.ylabel('Кол-во заказов')
plt.show()

In [ ]:
week_1 = decomposed.seasonal['2018-08-13':'2018-08-20']
week_1.plot(color='purple', figsize=(20,8))
plt.title('Сезонность')
plt.xlabel('Дата/время')
plt.ylabel('Кол-во заказов')
plt.show()

In [ ]:
week_2 = decomposed.resid['2018-08-13':'2018-08-20']
week_2.plot(color='red', figsize=(20,8))
plt.title('Остаток')
plt.xlabel('Дата/время')
plt.ylabel('Кол-во заказов')
plt.show()

**Построим графики для 15 августа 2018. Выбор этого дня связан с тем, что это середина недели, интересно посмотреть на динамику заказов**

In [ ]:
day = decomposed.trend['2018-08-15':'2018-08-16']
day.plot(color='yellowgreen', figsize=(20,8))
plt.title('Тренд')
plt.xlabel('Дата/время')
plt.ylabel('Кол-во заказов')
plt.show()

In [ ]:
day_1 = decomposed.seasonal['2018-08-15':'2018-08-16']
day_1.plot(color='hotpink', figsize=(20,8))
plt.title('Сезонность')
plt.xlabel('Дата/время')
plt.ylabel('Кол-во заказов')
plt.show()

In [ ]:
day_2 = decomposed.resid['2018-08-15':'2018-08-16']
day_2.plot(color='darkgreen', figsize=(20,8))
plt.title('Остаток')
plt.xlabel('Дата/время')
plt.ylabel('Кол-во заказов')
plt.show()

**Выводы по анализу и подготовке данных:**

* Пропусков нет, тип данных в столбце num_orders менять не надо
* Дубликаты искать бессмысленно, так как кол-во заказов из-за сезонности и времени суток может повторяться
* Рост количества заказов виден летом, что можно объяснить потоком туристов или, наоборот, тех, кто улетает или возвращается с отдыха
* Самый высокий спрос в августе
* В среднем больше всего заказов в понедельник
* В среднем пиковая нагрузка приходится на промежуток времени с 22:00 до 00:00
* В среднем меньше всего заказов между пятью и семью утра.
* Распределение сезонной части выглядит статическим в интервале суток.

## Обучение

**Введем функцию признаков:**

In [ ]:
def make_features(data, max_lag, rolling_mean_size):
    df = data.copy()
    df['day_of_week'] = df.index.day_name()
    df['hour'] = df.index.hour
    
    
    for lag in range(1, max_lag + 1):
        df['lag_{}'.format(lag)] = df['num_orders'].shift(lag)
        
        
    df['rolling_mean'] = df['num_orders'].shift().rolling(rolling_mean_size).mean()    
    return df

make_features(data, 24, 24)

In [ ]:
df = make_features(data, 24, 24)

In [ ]:
train, test = train_test_split(df, shuffle=False, test_size=0.2)
valid,test = train_test_split(test, shuffle = False, test_size = 0.5)
train = train.dropna()

### Модель линейной регрессии

In [ ]:
cat_features = ['day_of_week', 'hour']

In [ ]:
X_train = train.drop('num_orders', axis = 1)
y_train = train['num_orders']
X_valid = valid.drop('num_orders', axis = 1)
y_valid = valid['num_orders']

encoder_ohe = OneHotEncoder(drop='first',  handle_unknown='ignore', sparse=False)
encoder_ohe.fit(X_train[cat_features])

X_train[encoder_ohe.get_feature_names_out()] = encoder_ohe.transform(X_train[cat_features])
X_train = X_train.drop(cat_features, axis=1)

X_valid[encoder_ohe.get_feature_names_out()] = encoder_ohe.transform(X_valid[cat_features])
X_valid = X_valid.drop(cat_features, axis=1)

model_lm = LinearRegression()
model_lm.fit(X_train, y_train)

In [ ]:
pred_train = model_lm.predict(X_train)
print('RMSE на обучающей выборке = ', round(mean_squared_error(y_train, pred_train, squared = False), 2))
pred_valid = model_lm.predict(X_valid)
print('RMSE на валидационной выборке = ', round(mean_squared_error(y_valid, pred_valid, squared = False), 2))

**Обучим и проверим модель со сменой гиперпараметров**

In [ ]:
best_rmse_lm = 48
best_lag_lm = 0
best_roll_size_lm = 0
for lag in range(1, 168, 12):
    for roll_size in range(1, 168, 12):

        df = make_features(data, lag, roll_size)
        train, test = train_test_split(df, shuffle=False, test_size=0.2)
        valid,test = train_test_split(test, shuffle = False, test_size = 0.5)
        train = train.dropna()
        
        X_train = train.drop('num_orders', axis = 1)
        y_train = train['num_orders']
        X_valid = valid.drop('num_orders', axis = 1)
        y_valid = valid['num_orders']
        
        encoder_ohe = OneHotEncoder(drop='first',  handle_unknown='ignore', sparse=False)
        encoder_ohe.fit(X_train[cat_features])

        X_train[encoder_ohe.get_feature_names_out()] = encoder_ohe.transform(X_train[cat_features])
        X_train = X_train.drop(cat_features, axis=1)

        X_valid[encoder_ohe.get_feature_names_out()] = encoder_ohe.transform(X_valid[cat_features])
        X_valid = X_valid.drop(cat_features, axis=1)
        
        model_lm = LinearRegression()
        model_lm.fit(X_train, y_train)

        pred_valid = model_lm.predict(X_valid)
        rmse = mean_squared_error(y_valid, pred_valid, squared = False)
        
        if rmse < best_rmse_lm :
            best_rmse_lm = rmse
            best_lag_lm = lag
            best_roll_size_lm = roll_size
            best_y_valid_lm = y_valid
            best_preds_valid_lm = pred_valid
            model_lm1 = model_lm
        
print('Лучшая метрика RMSE на валид.выборке =', best_rmse_lm, 'при максимальном смещении', best_lag_lm,
      'с размером скользящего окна', best_roll_size_lm)

In [ ]:
fig, ax = plt.subplots(figsize=(20,8))
ax.plot(best_y_valid_lm, label='Исходная выборка')
ax.plot(best_y_valid_lm.index,best_preds_valid_lm, label='Предсказания')
plt.title('Распределение количества заказов по времени для валидационной выборки и предсказания')
plt.xlabel('Дата')
plt.ylabel('Кол-во заказов')
plt.legend()
plt.show()

**Графики показывают, что линейная регрессия достаточно точно предсказывает значения, за исключением всплесков на остатках**

### Дерево решений

In [ ]:
best_rmse_dt = 48
best_lag_dt = 0
best_roll_size_dt = 0
for lag in range(1, 168, 12):
    for roll_size in range(1, 168, 12):
        for max_depth in range(2, 21,12):

        
            df = make_features(data, lag,roll_size)
        train, test = train_test_split(df, shuffle=False, test_size=0.2)
        valid,test = train_test_split(test, shuffle = False, test_size = 0.5)
        train = train.dropna()
        
        X_train = train.drop('num_orders', axis = 1)
        y_train = train['num_orders']
        X_valid = valid.drop('num_orders', axis = 1)
        y_valid = valid['num_orders']
        
        encoder_ohe = OneHotEncoder(drop='first',  handle_unknown='ignore', sparse=False)
        encoder_ohe.fit(X_train[cat_features])

        X_train[encoder_ohe.get_feature_names_out()] = encoder_ohe.transform(X_train[cat_features])
        X_train = X_train.drop(cat_features, axis=1)

        X_valid[encoder_ohe.get_feature_names_out()] = encoder_ohe.transform(X_valid[cat_features])
        X_valid = X_valid.drop(cat_features, axis=1)
        
        model_dt = DecisionTreeRegressor(max_depth=max_depth, random_state=12345)
        model_dt.fit(X_train, y_train)

        pred_valid = model_dt.predict(X_valid)
        rmse = mean_squared_error(y_valid, pred_valid, squared = False)
        
        if rmse < best_rmse_dt :
            best_rmse_dt = rmse
            best_lag_dt = lag
            best_roll_size_dt = roll_size
            best_y_valid_dt = y_valid
            best_preds_valid_dt = pred_valid
            model_dt1 = model_dt
            best_max_depth = max_depth
            

print('Лучшая метрика RMSE на валид.выборке =', best_rmse_dt, 'при лаге', best_max_depth,
      'с размером скользящего окна', best_roll_size_dt, 'и глубине',best_max_depth )

In [ ]:
fig, ax = plt.subplots(figsize=(20,8))
ax.plot(best_y_valid_dt, label='Исходная выборка')
ax.plot(best_y_valid_dt.index,best_preds_valid_dt, label='Предсказания')
plt.title('Распределение количества заказов по времени для валидационной выборки и предсказания')
plt.xlabel('Дата')
plt.ylabel('Кол-во заказов')
plt.legend()
plt.show()

### Модель Lasso

In [ ]:
best_rmse_ls = 48
best_lag_ls = 0
best_roll_size_ls = 0
best_alpha_ls = 0
for lag in range(1, 168, 12):
    for roll_size in range(1, 168, 12):
        for alpha in np.arange(0.1, 1, 0.2):

            df = make_features(data, lag, roll_size)
            train, test = train_test_split(df, shuffle=False, test_size=0.2)
            valid,test = train_test_split(test, shuffle = False, test_size = 0.5)
            train = train.dropna()

            X_train = train.drop('num_orders', axis = 1)
            y_train = train['num_orders']
            X_valid = valid.drop('num_orders', axis = 1)
            y_valid = valid['num_orders']

            encoder_ohe = OneHotEncoder(drop='first',  handle_unknown='ignore', sparse_output=False)
            encoder_ohe.fit(X_train[cat_features])

            X_train[encoder_ohe.get_feature_names_out()] = encoder_ohe.transform(X_train[cat_features])
            X_train = X_train.drop(cat_features, axis=1)

            X_valid[encoder_ohe.get_feature_names_out()] = encoder_ohe.transform(X_valid[cat_features])
            X_valid = X_valid.drop(cat_features, axis=1)
 
            model_ls = Lasso(alpha=alpha)
            model_ls.fit(X_train, y_train)

            pred_valid = model_ls.predict(X_valid)
            rmse = mean_squared_error(y_valid, pred_valid, squared = False)
            if rmse < best_rmse_ls :
                best_rmse_ls = rmse
                best_lag_ls = lag
                best_roll_size_ls = roll_size
                best_alpha_ls = alpha
                best_y_valid_ls = y_valid
                best_preds_valid_ls = pred_valid
                model_ls1 = model_ls
                
print('Лучшая метрика RMSE на валид.выборке =', best_rmse_ls, 'при лаге', best_lag_ls,
      'с размером скользящего окна', best_roll_size_ls, 'и коэффициентом регуляризации',best_alpha_ls )

In [ ]:
fig, ax = plt.subplots(figsize=(20,8))
ax.plot(best_y_valid_ls, label='Исходная выборка')
ax.plot(best_y_valid_ls.index,best_preds_valid_ls, label='Предсказания')
plt.title('Распределение количества заказов по времени для валидационной выборки и предсказания')
plt.xlabel('Дата')
plt.ylabel('Кол-во заказов')
plt.legend()
plt.show()

**Выводы по результатам, полученным во время обучения моделей:**

* Были обучены 3 модели: линейная регрессия, дерево решений и Lasso 
* Лучшая метрика RMSE на валид.выборке у модели Lasso  и равна 28.9979964227437,следовательно,проведем ее тестирование на тестовой выборке с подобранными гиперпараметрами.

## Тестирование

**Лучшая метрика RMSE на валид.выборке у модели Lasso  и равна 28.9979964227437,следовательно,проведем ее тестирование на тестовой выборке с подобранными гиперпараметрами**

In [ ]:
lag_test = 157
roll_size_test = 1
alpha_test = 0.1

In [ ]:
df = make_features(data, lag_test, roll_size_test)
train, test = train_test_split(df, shuffle=False, test_size=0.2)
valid,test = train_test_split(test, shuffle = False, test_size = 0.5)
train = train.dropna()

X_train = train.drop('num_orders', axis = 1)
y_train = train['num_orders']
X_test = test.drop('num_orders', axis = 1)
y_test = test['num_orders']

encoder_ohe = OneHotEncoder(drop='first',  handle_unknown='ignore', sparse_output=False)
encoder_ohe.fit(X_train[cat_features])

X_train[encoder_ohe.get_feature_names_out()] = encoder_ohe.transform(X_train[cat_features])
X_train = X_train.drop(cat_features, axis=1)

X_test[encoder_ohe.get_feature_names_out()] = encoder_ohe.transform(X_test[cat_features])
X_test = X_test.drop(cat_features, axis=1)

model_lasso_t = Lasso(alpha=alpha)
model_lasso_t.fit(X_train, y_train)

pred_test = model_lasso_t.predict(X_test)
rmse = mean_squared_error(y_test, pred_test, squared = False)

print('RMSE лучшей модели на тестовой выборке = ', rmse)

In [ ]:
fig, ax = plt.subplots(figsize=(20,8))
ax.plot(y_test.index, y_test, label='Исходная выборка')
ax.plot(y_test.index, pred_test, label='Предсказания')
plt.title('Распределение количества заказов по времени для тестовой выборки и предсказания')
plt.xlabel('Дата')
plt.ylabel('Кол-во заказов')
plt.legend()
plt.show()

## Общий вывод

* Пропусков нет, тип данных в столбце num_orders менять не надо
* Дубликаты искать бессмысленно, так как кол-во заказов из-за сезонности и времени суток может повторяться
* Рост количества заказов виден летом, что можно объяснить потоком туристов или, наоборот, тех, кто улетает или возвращается с отдыха
* Самый высокий спрос в августе
* В среднем больше всего заказов в понедельник
* В среднем пиковая нагрузка приходится на промежуток времени с 22:00 до 00:00
* В среднем меньше всего заказов между пятью и семью утра.
* Распределение сезонной части выглядит статическим в интервале суток.
* Лучшая метрика RMSE на валид.выборке у модели Lasso  и равна 28.9979964227437
* RMSE лучшей модели на тестовой выборке =  39.16038099987398, что удовлетвуоряет условию ТЗ (RMSE < 48)